In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import pytorch_lightning as pl
from torch.utils.data import DataLoader, random_split
from torchvision import datasets, transforms
from tqdm import tqdm

2023-09-07 09:40:17.868599: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-07 09:40:18.833794: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
import torchmetrics
from torchmetrics import Metric

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [4]:
class NN(pl.LightningModule):
    def __init__(self, input_size, num_classes):
        super().__init__()
        self.fc1        = nn.Linear(input_size, 50)
        self.fc2        = nn.Linear(50, num_classes)
        self.loss_fn    = nn.CrossEntropyLoss()
        self.accuracy   = torchmetrics.Accuracy(task='multiclass', num_classes=10)
        self.f1_score   = torchmetrics.F1Score(task='multiclass', num_classes=10, average='macro')


    def forward(self, x):
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        return x

    # We can use _common_step() to reduce code duplication
    def _common_step(self, batch, batch_idx):
        x, y    = batch
        # Flatten image
        x       = x.reshape(x.shape[0], -1)
        logits  = self.forward(x)
        loss    = F.cross_entropy(logits, y)
        return loss, logits, y

    def training_step(self, batch, batch_idx):
        loss, logits, y = self._common_step(batch, batch_idx)
        accuracy = self.accuracy(logits, y)     # Taking more computation time
        f1_score = self.f1_score(logits, y)     # Taking more computation time
        self.log_dict(
            {'train_loss': loss, 'train_accuracy': accuracy, 'train_f1_score': f1_score},
            on_step=False, on_epoch=True, prog_bar=True, logger=True)
        return loss

    def validation_step(self, batch, batch_idx):
        loss, logits, y = self._common_step(batch, batch_idx)
        self.log('val_loss', loss)
        return loss

    def test_step(self, batch, batch_idx):
        loss, logits, y = self._common_step(batch, batch_idx)
        self.log('test_loss', loss)
        return loss


    def predict_step(self, batch, batch_idx):
        x, y    = batch
        # Flatten image
        x       = x.reshape(x.shape[0], -1)
        logits  = self.forward(x)
        preds   = torch.argmax(logits, dim=1)
        return preds

    def configure_optimizers(self):
        return optim.Adam(self.parameters(), lr=1e-3)

In [5]:
entire_dataset = datasets.MNIST(
    root='dataset/',
    train=True,
    transform=transforms.ToTensor(),
    download=True
)

100%|██████████| 9912422/9912422 [00:02<00:00, 4770979.27it/s]


Extracting dataset/MNIST/raw/train-images-idx3-ubyte.gz to dataset/MNIST/raw



100%|██████████| 28881/28881 [00:00<00:00, 56871217.76it/s]

Extracting dataset/MNIST/raw/train-labels-idx1-ubyte.gz to dataset/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 4439598.68it/s]


Extracting dataset/MNIST/raw/t10k-images-idx3-ubyte.gz to dataset/MNIST/raw



100%|██████████| 4542/4542 [00:00<00:00, 13626987.67it/s]

Extracting dataset/MNIST/raw/t10k-labels-idx1-ubyte.gz to dataset/MNIST/raw



In [6]:
train_ds, val_ds = random_split(entire_dataset, [55000, 5000])
test_ds = datasets.MNIST(
    root='dataset/',
    train=False,
    transform=transforms.ToTensor(),
    download=True
)

In [7]:
train_loader = DataLoader(dataset=train_ds,
                          batch_size=32,
                          num_workers=4,
                          shuffle=True)

val_loader = DataLoader(dataset=val_ds,
                        batch_size=32,
                        num_workers=4,
                        shuffle=False)

test_loader = DataLoader(dataset=test_ds,
                        batch_size=32,
                        num_workers=4,
                        shuffle=False)

In [8]:
model = NN(input_size=784, num_classes=10).to(device)

In [9]:
trainer = pl.Trainer(min_epochs=2, max_epochs=5, devices=[0], accelerator='gpu', precision=16)
trainer.fit(model, train_loader, val_loader)

/home/longln/.local/lib/python3.8/site-packages/lightning_fabric/connector.py:554: UserWarning: 16 is supported for historical reasons but its usage is discouraged. Please set your precision to 16-mixed instead!
  rank_zero_warn(
Using 16bit Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA GeForce RTX 3050 Laptop GPU') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
Missing logger folder: /home/longln/Source-Code/PyTorchLinghtning-Tutorial/PyTorchLightning#4.Metrics/lightning_logs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name     | Type               | Par

Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_epochs=5` reached.


In [10]:
trainer.validate(model, val_loader)
trainer.test(model, test_loader)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Validation: 0it [00:00, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃      Validate metric      ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         val_loss          │    0.11585549265146255    │
└───────────────────────────┴───────────────────────────┘

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: 0it [00:00, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_loss         │    0.09803919494152069    │
└───────────────────────────┴───────────────────────────┘

[{'test_loss': 0.09803919494152069}]

In [11]:
# Custom Metric (Accuracy)
class MyAccuracy(Metric):
    def __init__(self):
        super().__init__()
        self.add_state("correct", default=torch.tensor(0), dist_reduce_fx="sum")
        self.add_state("total", default=torch.tensor(0), dist_reduce_fx="sum")

    def update(self, predicitons, targets):
        predictions     = torch.argmax(predicitons, dim=1)
        assert predictions.shape == targets.shape
        self.correct    += torch.sum(predictions == targets)
        self.total      += targets.numel()

    def compute(self):
        return self.correct.float() / self.total.float()